Copyright (c) Microsoft Corporation. All rights reserved.  
Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/NotebookVM/how-to-use-azureml/machine-learning-pipelines/intro-to-pipelines/aml-pipelines-with-automated-machine-learning-step.png)

# Azure Machine Learning Pipeline with AutoMLStep (Udacity Course 2)
This notebook demonstrates the use of AutoMLStep in Azure Machine Learning Pipeline.

## Introduction
In this example we showcase how you can use AzureML Dataset to load data for AutoML via AML Pipeline. 

If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, make sure you have executed the [configuration](https://aka.ms/pl-config) before running this notebook.

In this notebook you will learn how to:
1. Create an `Experiment` in an existing `Workspace`.
2. Create or Attach existing AmlCompute to a workspace.
3. Define data loading in a `TabularDataset`.
4. Configure AutoML using `AutoMLConfig`.
5. Use AutoMLStep
6. Train the model using AmlCompute
7. Explore the results.
8. Test the best fitted model.

## Azure Machine Learning and Pipeline SDK-specific imports

In [32]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.22.0


## Initialize Workspace
Initialize a workspace object from persisted configuration. Make sure the config file is present at .\config.json

In [33]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-140435
aml-quickstarts-140435
southcentralus
61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30


## Create an Azure ML experiment
Let's create an experiment named "automlstep-classification" and a folder to hold the training scripts. The script runs will be recorded under the experiment in Azure.

The best practice is to use separate folders for scripts and its dependent files for each step and specify that folder as the `source_directory` for the step. This helps reduce the size of the snapshot created for the step (only the specific folder is snapshotted). Since changes in any files in the `source_directory` would trigger a re-upload of the snapshot, this helps keep the reuse of the step when there are no changes in the `source_directory` of the step.

*Udacity Note:* There is no need to create an Azure ML experiment, this needs to re-use the experiment that was already created


In [34]:
from azureml.core import Workspace
from azureml.core.webservice import Webservice

# Choose a name for the run history container in the workspace.
# NOTE: update these to match your existing experiment name
experiment_name = 'udacity-project-2'
project_folder = './pipeline-project'

experiment = Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
udacity-project-2,quick-starts-ws-140435,Link to Azure Machine Learning studio,Link to Documentation


### Create or Attach an AmlCompute cluster
You will need to create a [compute target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target) for your AutoML run. In this tutorial, you get the default `AmlCompute` as your training compute resource.

**Udacity Note** There is no need to create a new compute target, it can re-use the previous cluster

In [35]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "auto-ml"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=5)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True) #min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Data

**Udacity note:** Make sure the `key` is the same name as the dataset that is uploaded, and that the description matches. If it is hard to find or unknown, loop over the `ws.datasets.keys()` and `print()` them.
If it *isn't* found because it was deleted, it can be recreated with the link that has the CSV 

In [36]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "BankMarketing Dataset"
description_text = "Bank Marketing DataSet for Udacity Course 2"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
count,32950.00,32950.00,32950.00,32950.00,32950.00,32950.00,32950.00,32950.00,32950.00,32950.00
mean,40.04,257.34,2.56,962.17,0.17,0.08,93.57,-40.52,3.62,5166.86
std,10.43,257.33,2.76,187.65,0.50,1.57,0.58,4.62,1.74,72.21
min,17.00,0.00,1.00,0.00,0.00,-3.40,92.20,-50.80,0.63,4963.60
25%,32.00,102.00,1.00,999.00,0.00,-1.80,93.08,-42.70,1.34,5099.10
50%,38.00,179.00,2.00,999.00,0.00,1.10,93.75,-41.80,4.86,5191.00
75%,47.00,318.00,3.00,999.00,0.00,1.40,93.99,-36.40,4.96,5228.10
max,98.00,4918.00,56.00,999.00,7.00,1.40,94.77,-26.90,5.04,5228.10


### Review the Dataset Result

You can peek the result of a TabularDataset at any range using `skip(i)` and `take(j).to_pandas_dataframe()`. Doing so evaluates only `j` records for all the steps in the TabularDataset, which makes it fast even against large datasets.

`TabularDataset` objects are composed of a list of transformation steps (optional).

In [37]:
dataset.take(5).to_pandas_dataframe()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,57,technician,married,high.school,no,no,yes,cellular,may,mon,...,1,999,1,failure,-1.80,92.89,-46.20,1.30,5099.10,no
1,55,unknown,married,unknown,unknown,yes,no,telephone,may,thu,...,2,999,0,nonexistent,1.10,93.99,-36.40,4.86,5191.00,no
2,33,blue-collar,married,basic.9y,no,no,no,cellular,may,fri,...,1,999,1,failure,-1.80,92.89,-46.20,1.31,5099.10,no
3,36,admin.,married,high.school,no,no,no,telephone,jun,fri,...,4,999,0,nonexistent,1.40,94.47,-41.80,4.97,5228.10,no
4,27,housemaid,married,high.school,no,yes,no,cellular,jul,fri,...,2,999,0,nonexistent,1.40,93.92,-42.70,4.96,5228.10,no


## Train
This creates a general AutoML settings object.
**Udacity notes:** These inputs must match what was used when training in the portal. `label_column_name` has to be `y` for example.

In [38]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             blocked_models = ['XGBoostClassifier'],
                             training_data=dataset,
                             label_column_name="y",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

#### Create Pipeline and AutoMLStep

You can define outputs for the AutoMLStep using TrainingOutput.

In [39]:
from azureml.pipeline.core import PipelineData, TrainingOutput

ds = ws.get_default_datastore()
metrics_output_name = 'metrics_output'
best_model_output_name = 'best_model_output'

metrics_data = PipelineData(name='metrics_data',
                           datastore=ds,
                           pipeline_output_name=metrics_output_name,
                           training_output=TrainingOutput(type='Metrics'))
model_data = PipelineData(name='model_data',
                           datastore=ds,
                           pipeline_output_name=best_model_output_name,
                           training_output=TrainingOutput(type='Model'))

Create an AutoMLStep.

In [40]:
automl_step = AutoMLStep(
    name='automl_module',
    automl_config=automl_config,
    outputs=[metrics_data, model_data],
    allow_reuse=True)

In [41]:
from azureml.pipeline.core import Pipeline
pipeline = Pipeline(
    description="pipeline_with_automlstep",
    workspace=ws,    
    steps=[automl_step])

In [42]:
pipeline_run = experiment.submit(pipeline)

Created step automl_module [d42ef295][04cabeac-0c81-4222-ae51-24da1aacf447], (This step will run and generate new outputs)
Submitted PipelineRun 3157c6e1-48b2-483a-9aa6-fba7a6cdc09f
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/udacity-project-2/runs/3157c6e1-48b2-483a-9aa6-fba7a6cdc09f?wsid=/subscriptions/61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30/resourcegroups/aml-quickstarts-140435/workspaces/quick-starts-ws-140435


In [43]:
from azureml.widgets import RunDetails
RunDetails(pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

In [44]:
pipeline_run.wait_for_completion()

PipelineRunId: 3157c6e1-48b2-483a-9aa6-fba7a6cdc09f
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/udacity-project-2/runs/3157c6e1-48b2-483a-9aa6-fba7a6cdc09f?wsid=/subscriptions/61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30/resourcegroups/aml-quickstarts-140435/workspaces/quick-starts-ws-140435
PipelineRun Status: Running


StepRunId: 4d6695a0-663e-49ac-841f-da9de59f6059
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/udacity-project-2/runs/4d6695a0-663e-49ac-841f-da9de59f6059?wsid=/subscriptions/61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30/resourcegroups/aml-quickstarts-140435/workspaces/quick-starts-ws-140435
StepRun( automl_module ) Status: Running

StepRun(automl_module) Execution Summary
StepRun( automl_module ) Status: Finished
{'runId': '4d6695a0-663e-49ac-841f-da9de59f6059', 'target': 'auto-ml', 'status': 'Completed', 'startTimeUtc': '2021-03-14T21:50:51.571004Z', 'endTimeUtc': '2021-03-14T22:13:30.811233Z', 'properties': {'ContentSnapshotId': '

'Finished'

## Examine Results

### Retrieve the metrics of all child runs
Outputs of above run can be used as inputs of other steps in pipeline. In this tutorial, we will examine the outputs by retrieve output data and running some tests.

In [45]:
metrics_output = pipeline_run.get_pipeline_output(metrics_output_name)
num_file_downloaded = metrics_output.download('.', show_progress=True)

Downloaded azureml/4d6695a0-663e-49ac-841f-da9de59f6059/metrics_data, 1 files out of an estimated total of 1


In [46]:
import json
with open(metrics_output._path_on_datastore) as f:
    metrics_output_result = f.read()
    
deserialized_metrics_output = json.loads(metrics_output_result)
df = pd.DataFrame(deserialized_metrics_output)
df

,4d6695a0-663e-49ac-841f-da9de59f6059_2,4d6695a0-663e-49ac-841f-da9de59f6059_0,4d6695a0-663e-49ac-841f-da9de59f6059_8,4d6695a0-663e-49ac-841f-da9de59f6059_14,4d6695a0-663e-49ac-841f-da9de59f6059_6,4d6695a0-663e-49ac-841f-da9de59f6059_1,4d6695a0-663e-49ac-841f-da9de59f6059_7,4d6695a0-663e-49ac-841f-da9de59f6059_4,4d6695a0-663e-49ac-841f-da9de59f6059_9,4d6695a0-663e-49ac-841f-da9de59f6059_11,...,4d6695a0-663e-49ac-841f-da9de59f6059_30,4d6695a0-663e-49ac-841f-da9de59f6059_32,4d6695a0-663e-49ac-841f-da9de59f6059_37,4d6695a0-663e-49ac-841f-da9de59f6059_35,4d6695a0-663e-49ac-841f-da9de59f6059_34,4d6695a0-663e-49ac-841f-da9de59f6059_42,4d6695a0-663e-49ac-841f-da9de59f6059_36,4d6695a0-663e-49ac-841f-da9de59f6059_15,4d6695a0-663e-49ac-841f-da9de59f6059_16,4d6695a0-663e-49ac-841f-da9de59f6059_41
recall_score_weighted,[0.7820940819423369],[0.9116843702579667],[0.7189681335356601],[0.703793626707132],[0.891350531107739],[0.8880121396054628],[0.8880121396054628],[0.8995447647951441],[0.8154779969650986],[0.7405159332321699],...,[0.9119878603945372],[0.8898330804248862],[0.9059180576631259],[0.8880121396054628],[0.9040971168437025],[0.8880121396054628],[0.9077389984825494],[0.7468892261001517],[0.716236722306525],[0.9147192716236723]
average_precision_score_micro,[0.8561029560400457],[0.9806603102489483],[0.8100974399656617],[0.7951942030905311],[0.9610168024788672],[0.9684237493518866],[0.9645700553359331],[0.9701327493873848],[0.8620732238023707],[0.841095975786235],...,[0.9794493482917321],[0.9745732685613119],[0.9777437570164424],[0.9735304728652626],[0.9685634185250935],[0.9777124510713131],[0.9779096405693076],[0.798829018945425],[0.8417351232845661],[0.9809463385616601]
weighted_accuracy,[0.7758005996341305],[0.9514937218005303],[0.716116227890403],[0.6992955672902882],[0.9839297731096389],[0.9843450583187134],[0.9843450583187134],[0.9765496076693014],[0.81251206505859],[0.7379436506720158],...,[0.9547700139704612],[0.9843056223201774],[0.960095253457922],[0.9843450583187134],[0.9616046289354224],[0.9843450583187134],[0.9553520111267004],[0.7491241586651639],[0.7130885089823823],[0.9542698781289677]
accuracy,[0.7820940819423369],[0.9116843702579667],[0.7189681335356601],[0.703793626707132],[0.891350531107739],[0.8880121396054628],[0.8880121396054628],[0.8995447647951441],[0.8154779969650986],[0.7405159332321699],...,[0.9119878603945372],[0.8898330804248862],[0.9059180576631259],[0.8880121396054628],[0.9040971168437025],[0.8880121396054628],[0.9077389984825494],[0.7468892261001517],[0.716236722306525],[0.9147192716236723]
f1_score_macro,[0.663334175777012],[0.7653697272147331],[0.5957785790216135],[0.5838431898695389],[0.507417076731375],[0.4703423886834914],[0.4703423886834914],[0.621573054448415],[0.6961291105448368],[0.6160589038805209],...,[0.7597246696921418],[0.4893546986488927],[0.7199819069027907],[0.4703423886834914],[0.7067659940700954],[0.4703423886834914],[0.7405646205646206],[0.6157353804849399],[0.5936082997787209],[0.7716861704790277]
f1_score_micro,[0.7820940819423369],[0.9116843702579667],[0.7189681335356602],[0.703793626707132],[0.891350531107739],[0.8880121396054628],[0.8880121396054628],[0.8995447647951441],[0.8154779969650986],[0.7405159332321699],...,[0.9119878603945372],[0.8898330804248862],[0.9059180576631259],[0.8880121396054628],[0.9040971168437025],[0.8880121396054628],[0.9077389984825494],[0.7468892261001517],[0.7162367223065251],[0.9147192716236722]
norm_macro_recall,[0.6148862548092331],[0.5026785366965085],[0.4609102208588729],[0.4438220458759612],[0.036915088904819315],[0.0],[0.0],[0.17876546502990665],[0.6548485033722518],[0.5017532745388973],...,[0.4793376641900391],[0.018628426202238924],[0.3754035865717509],[0.0],[0.3449338423664483],[0.0],[0.4319242303837938],[0.47577461762314144],[0.4578343493619488],[0.5108326988943164]
balanced_accuracy,[0.8074431274046165],[0.7513392683482543],[0.7304551104294364],[0.7219110229379806],[0.5184575444524097],[0.5],[0.5],[0.5893827325149533],[0.8274242516861259],[0

### Retrieve the Best Model

In [47]:
# Retrieve best model from Pipeline Run
best_model_output = pipeline_run.get_pipeline_output(best_model_output_name)
num_file_downloaded = best_model_output.download('.', show_progress=True)

Downloaded azureml/4d6695a0-663e-49ac-841f-da9de59f6059/model_data, 1 files out of an estimated total of 1


In [48]:
import pickle

with open(best_model_output._path_on_datastore, "rb" ) as f:
    best_model = pickle.load(f)
best_model

PipelineWithYTransformations(Pipeline={'memory': None,
                                       'steps': [('datatransformer',
                                                  DataTransformer(enable_dnn=None,
                                                                  enable_feature_sweeping=None,
                                                                  feature_sweeping_config=None,
                                                                  feature_sweeping_timeout=None,
                                                                  featurization_config=None,
                                                                  force_text_dnn=None,
                                                                  is_cross_validation=None,
                                                                  is_onnx_compatible=None,
                                                                  logger=None,
                                                              

In [49]:
best_model.steps

[('datatransformer',
  DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                  feature_sweeping_config=None, feature_sweeping_timeout=None,
                  featurization_config=None, force_text_dnn=None,
                  is_cross_validation=None, is_onnx_compatible=None, logger=None,
                  observer=None, task=None, working_dir=None)),
 ('prefittedsoftvotingclassifier',
  PreFittedSoftVotingClassifier(classification_labels=None,
                                estimators=[('22',
                                             Pipeline(memory=None,
                                                      steps=[('maxabsscaler',
                                                              MaxAbsScaler(copy=True)),
                                                             ('lightgbmclassifier',
                                                              LightGBMClassifier(boosting_type='goss',
                                                         

### Test the Model
#### Load Test Data
For the test data, it should have the same preparation step as the train data. Otherwise it might get failed at the preprocessing step.

In [50]:
dataset_test = Dataset.Tabular.from_delimited_files(path='https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv')
df_test = dataset_test.to_pandas_dataframe()
df_test = df_test[pd.notnull(df_test['y'])]

y_test = df_test['y']
X_test = df_test.drop(['y'], axis=1)

#### Testing Our Best Fitted Model

We will use confusion matrix to see how our model works.

In [51]:
from sklearn.metrics import confusion_matrix
ypred = best_model.predict(X_test)
cm = confusion_matrix(y_test, ypred)

In [52]:
# Visualize the confusion matrix
pd.DataFrame(cm).style.background_gradient(cmap='Blues', low=0, high=0.9)

,0,1
0,28400,858
1,1309,2383


## Publish and run from REST endpoint

Run the following code to publish the pipeline to your workspace. In your workspace in the portal, you can see metadata for the pipeline including run history and durations. You can also run the pipeline manually from the portal.

Additionally, publishing the pipeline enables a REST endpoint to rerun the pipeline from any HTTP library on any platform.


In [53]:
published_pipeline = pipeline_run.publish_pipeline(
    name="Bankmarketing Train", description="Training bankmarketing pipeline", version="1.0")

published_pipeline


Name,Id,Status,Endpoint
Bankmarketing Train,468c4d8a-24fe-4e61-95c0-716ee964e32e,Active,REST Endpoint


Authenticate once again, to retrieve the `auth_header` so that the endpoint can be used

In [54]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()



Get the REST url from the endpoint property of the published pipeline object. You can also find the REST url in your workspace in the portal. Build an HTTP POST request to the endpoint, specifying your authentication header. Additionally, add a JSON payload object with the experiment name and the batch size parameter. As a reminder, the process_count_per_node is passed through to ParallelRunStep because you defined it is defined as a PipelineParameter object in the step configuration.

Make the request to trigger the run. Access the Id key from the response dict to get the value of the run id.


In [55]:
import requests

rest_endpoint = published_pipeline.endpoint
response = requests.post(rest_endpoint, 
                         headers=auth_header, 
                         json={"ExperimentName": "pipeline-rest-endpoint"}
                        )

In [56]:
try:
    response.raise_for_status()
except Exception:    
    raise Exception("Received bad response from the endpoint: {}\n"
                    "Response Code: {}\n"
                    "Headers: {}\n"
                    "Content: {}".format(rest_endpoint, response.status_code, response.headers, response.content))

run_id = response.json().get('Id')
print('Submitted pipeline run: ', run_id)

Submitted pipeline run:  7e1726e1-ce7a-474b-85fe-d07f23e3efa1


Use the run id to monitor the status of the new run. This will take another 10-15 min to run and will look similar to the previous pipeline run, so if you don't need to see another pipeline run, you can skip watching the full output.

In [57]:
from azureml.pipeline.core.run import PipelineRun
from azureml.widgets import RunDetails

published_pipeline_run = PipelineRun(ws.experiments["pipeline-rest-endpoint"], run_id)
RunDetails(published_pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …